In [5]:
import numpy as np
import pandas as pd
from sympy import Symbol, diff, cos, sin, exp, tan
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-talk')

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets


In [24]:
line = lambda x, x0, x1, y0, y1 : ((y1-y0)/(x1-x0))*(x-x0) + y0
tangente = lambda x, m, b : m * x + b

def numericalDer(f, x0, h, r):
    t = Symbol('x')
    fp = diff(f,t)
    evalfp = lambdify(t, fp, modules=['numpy'])    
    evalf = lambdify(t, f, modules=['numpy']) # Evalua la función
   
    offset = r * np.pi
    xl = np.linspace(x0 - offset*0.25, x0 + h + offset*0.25, 50)  # línea tangente aprox
    xt = np.linspace(x0 - offset*0.25, x0 + offset*0.25, 50)  # línea tangente
    xv = np.linspace(x0 - offset, x0 + offset, 50) # funcion f(x)
    
    lf = line(xl, x0, x0 + h, evalf(x0), evalf(x0 + h))

    yv = evalf(xv)
    yp = evalfp(xv)

    f_0 = evalf(x0)
    fph = evalf(x0+h)
    fp_0 = evalfp(x0)
    
    lt = tangente(xt, fp_0, f_0 - fp_0 * x0)
    
    ancho_linea = 2.0
    
    fig = plt.figure(figsize=(20,5))

    #----------------- Gráfica de la función original -----------------
    ax1 = plt.subplot(1,2,1)
    
    # Función original
    ax1.plot(xv, yv, '-', lw = 3, color='k', label = 'f(x)')

    # Línea tangente
    ax1.scatter(x0, f_0, facecolor ='C1', edgecolor='k', zorder=10, label='$x_o$')
    ax1.scatter(x0+h, fph, facecolor ='w', edgecolor='k', zorder=5)  
    ax1.plot(xl, lf, c='C2', lw = ancho_linea, label="$F'(x)$", zorder=2)
    ax1.plot(xt, lt, c='C3', lw = 4.0, zorder=1)

#    plt.ylim(-1.2, 1.2)
    
    # Líneas punteadas verdes
    ax1.plot([x0, x0],[0, f_0],'g--', lw=1)
    ax1.plot([x0+h, x0+h],[0, fph],'g--', lw=1)
    ax1.plot([0, x0], [f_0, f_0], 'g--', lw=1)
    ax1.plot([0, x0+h], [fph, fph], 'g--', lw=1)
    
    # Tics de la gráfica
    plt.xticks(ticks=[x0, x0+h], labels=['$x_o$', '$x_o+h$'])
    plt.yticks(ticks=[f_0, fph], labels=['$f(x_o)$', '$f(x_o+h)$'])
    
    # Otras decoraciones
    ax1.set_title("Función $f(x)$ y su derivada aproximada $F'(x)$", c='b')
    ax1.legend(loc='upper right', 
               ncol=1, framealpha=0.99)
    ax1.spines['left'].set_position('zero')
    ax1.spines['right'].set_color('none')
    ax1.spines['bottom'].set_position('zero')
    ax1.spines['top'].set_color('none')
    a_dx = (fph - f_0) / h
    text_fun = " $f(x_o)$ = {:<5.4f} \t $f(x_o+h)$ = {:<5.4f} \t $F'(x_o)$ = {:<5.4f}".format(f_0, fph, a_dx)
    ax1.text(x=0.15, y=0.0, s=text_fun, c='b', fontsize=12, 
             transform=ax1.transAxes, bbox=dict(facecolor='yellow', alpha=0.99))
        
    #----------------- Gráfica de la derivada de la función -----------------    
    ax2 = plt.subplot(1,2,2)
    
    # Derivada de la función
    ax2.scatter(x0, fp_0, facecolor ='C1', edgecolor='k', zorder=10)
    ax2.plot(xv, yp, '-', c='C2', lw = 3, label = "$\dfrac{df(x)}{dx}$")
#    plt.ylim(-1.2, 1.2)
    ax2.plot(xl, lf, c='white', lw = ancho_linea, zorder=0)


    # Líneas punteadas verdes
    ax2.plot([x0, x0],[0, fp_0],'g--', lw=1)
    ax2.plot([0, x0], [fp_0, fp_0], 'g--', lw=1)
    
    # Tics de la gráfica
    plt.xticks(ticks=[x0], 
                   labels=['$x_o$'])
    plt.yticks(ticks=[fp_0], labels=["$f\, '(x_o)$"])

    # Otras decoraciones
    ax2.set_title('Derivada exacta de $f(x)$', c='b')
    ax2.legend(loc='upper right', 
               ncol=1, framealpha=0.99)
    ax2.spines['left'].set_position('zero')
    ax2.spines['right'].set_color('none')
    ax2.spines['bottom'].set_position('zero')
    ax2.spines['top'].set_color('none')
    
    text_fun = " $f'(x_o)$ = {:<5.4f} \t $|f'(x_o) - F'(x_o)|$ = {:<5.4f}".format(fp_0, np.fabs(a_dx - fp_0))
    ax2.text(x=0.25, y=0.0, s=text_fun, c='b', fontsize=12, 
             transform=ax2.transAxes, bbox=dict(facecolor='yellow', alpha=0.99))
    
    plt.show()

In [25]:
t = Symbol('x')
lista_funciones = [sin(t), cos(t), t**2 * sin(t)]

w = interact(numericalDer,
             f = widgets.Dropdown(options=lista_funciones, value=sin(t), description='Función'), 
             x0 = widgets.FloatSlider(min=0, max=np.pi, step=0.5, value=np.pi * 0.5, description='$x_o$'),
             h = widgets.FloatSlider(min=0.001, max=1.001, step=0.1, value=1.0),
             r = fixed(1.25))

display(w)

interactive(children=(Dropdown(description='Función', options=(sin(x), cos(x), x**2*sin(x)), value=sin(x)), Fl…

<function __main__.numericalDer(f, x0, h, r)>